<a href="https://colab.research.google.com/github/DIPANJAN001/Forecasting-Solar-Energy/blob/master/gms1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install Boruta
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import concatenate
from keras import Model
from keras.layers import Input
from keras.layers import Bidirectional
from keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.layers import Input
from sklearn.decomposition import PCA 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 7.9 MB/s eta 0:00:00


In [2]:
def lstm_data_transform(x_data, y_data, num_steps):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [3]:
df=pd.read_excel("/content/pv_04.xlsx")
weather_input1=df.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
solpow=df['power_normed']

In [4]:
rfc = RandomForestRegressor(random_state=1, n_estimators=1000, max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1)
boruta_selector.fit(np.array(weather_input), np.array(solpow)) 

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	49
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	9
Tentative: 	17
Rejected: 	23
Iteration: 	9 / 100
Confirmed: 	9
Tentative: 	17
Rejected: 	23
Iteration: 	10 / 100
Confirmed: 	9
Tentative: 	17
Rejected: 	23
Iteration: 	11 / 100
Confirmed: 	9
Tentative: 	17
Rejected: 	23
Iteration: 	12 / 100
Confirmed: 	10
Tentative: 	16
Rejected: 	23
Iteration: 	13 / 100
Confirmed: 	10
Tentative: 	16
Rejected: 	23
Iteration: 	14 / 100
Confirmed: 	10
Tentative: 	16
Rejected: 	23
Iteration: 	15 / 100
Confirmed: 	10
Tentative: 	16
Rejected: 	23
Iteration: 	16 / 100
Confirmed: 	11
Tentative: 	15
R

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=141,
                                         random_state=RandomState(MT19937) at 0x7F6909607440),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7F6909607440, verbose=2)

In [5]:
X_important_train = boruta_selector.transform(np.array(weather_input))
num_steps = 2
# training set
(x_transformed_train,
 y_transformed_train) = lstm_data_transform(X_important_train,solpow , num_steps=num_steps)
assert x_transformed_train.shape[0] == y_transformed_train.shape[0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_transformed_train,y_transformed_train,test_size=0.33, random_state=42,shuffle=False)
#X_train_,X_val,y_train_,y_val=train_test_split(X_train,y_train,test_size=0.2, random_state=42,shuffle=False)
inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))

In [6]:
inputs1

<KerasTensor: shape=(None, 2, 18) dtype=float32 (created by layer 'input_1')>

In [7]:
from keras import optimizers

In [137]:
opt=optimizers.Adam(learning_rate=0.003)

In [9]:
from keras.layers import Bidirectional

In [94]:
from keras import optimizers

class CustomAdam(optimizers.Adam):
    def __init__(self, new_idea_param=0.1, *args, **kwargs):
        self.new_idea_param = new_idea_param
        super(CustomAdam, self).__init__(*args, **kwargs)

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t)))

        new_idea_t = self.new_idea_param * (K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - new_idea_t * g

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            self.updates.append(K.update(p, p_t))
        return self.updates

In [95]:
from keras import optimizers

class GradientAdam(optimizers.Adam):
    def __init__(self, gradient_param=0.1, *args, **kwargs):
        self.gradient_param = gradient_param
        super(GradientAdam, self).__init__(*args, **kwargs)

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t)))

        gradient_t = self.gradient_param * grads

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v, g_t in zip(params, grads, ms, vs, gradient_t):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) + g_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            self.updates.append(K.update(p, p_t))
        return self.updates


In [105]:
from tensorflow.keras.optimizers import Adam
from tensorflow.python.ops import math_ops

class HalvAdam(Adam):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.prev_gradients = None

    @tf.function
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [math_ops.cast(x, "float32") for x in grads]

        if self.prev_gradients is not None:
            for i in range(len(grads)):
                if (grads[i] * self.prev_gradients[i] < 0):
                    self.updates[i] = self.updates[i] / 2

        self.prev_gradients = grads
        return self.updates

In [123]:
from tensorflow.keras.optimizers import Adam
from tensorflow.python.ops import math_ops

class AdaptiveAdam(Adam):
    def __init__(self, *args, factor=0.5, patience=5, **kwargs):
        super().__init__(*args, **kwargs)
        self.factor = factor
        self.patience = patience
        self.wait = 0
        self.best_loss = float('inf')
        self.best_weights = None

    @tf.function
    def get_updates(self, loss, params):
        current_loss = loss()
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.best_weights = params
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.wait = 0
                self.lr = self.lr * self.factor
                params = self.best_weights
        grads = self.get_gradients(loss, params)
        self.updates = [math_ops.cast(g, "float32") for g in grads]
        return self.updates


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.python.ops import math_ops

class MomentumAdam(Adam):
    def __init__(self, *args, momentum=0.9, **kwargs):
        super().__init__(*args, **kwargs)
        self.momentum = momentum
        self.velocities = [tf.Variable(tf.zeros_like(p), trainable=False) for p in self.weights]

    @tf.function
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = []
        for p, g, v in zip(params, grads, self.velocities):
            v_t = self.momentum * v - self.lr * g
            p_t = p + v_t
            self.updates.append(p_t)
            self.updates.append(v_t)
        return self.updates


In [96]:
from keras import backend as K

In [97]:
import tensorflow as tf

In [155]:
def define_model():

  fe1_2 = Bidirectional(LSTM(64, activation='relu',return_sequences = True))(inputs1)
  fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=Bidirectional(LSTM(8, activation='relu'))(fe1_3)
  out1_1=Dense(1, activation='relu')(fe1_4)


  fe2_2 = Bidirectional(LSTM(32, activation='relu',return_sequences = True))(inputs1)
  fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=Bidirectional(LSTM(8, activation='relu'))(fe2_3)
  out2_1=Dense(1, activation='relu')(fe2_4)

  fe3_2 = Bidirectional(LSTM(96, activation='relu',return_sequences = True))(inputs1)
  fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_3)#16
  out3_1=Dense(1, activation='relu')(fe3_4)
 
 

  output = layers.average([out2_1, out3_1])
  #merged3 = concatenate([out2_1,out3_1], name='concat3')
  #output = Dense(1, activation='relu')( merged3)
  

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer=opt,loss='mean_squared_error',metrics=['mse'])
 
  return model
mdl=define_model()


In [156]:
loss=[]

In [157]:
from keras.models import clone_model

In [158]:
from keras.optimizers import Adam

In [159]:
from keras.callbacks import ModelCheckpoint

In [160]:
def GlobalMinimaSearch(weights):
  if len(loss)>5:
   return
  
  initial_weights =weights
  model=clone_model(mdl)
  model.set_weights(weights)
  model.compile(optimizer=AdaptiveAdam(), loss='mean_squared_error')
  model.fit(X_train, y_train, epochs=75, batch_size=64)
  y= model.predict(X_test)
  loss.append(np.sqrt(mean_squared_error(y,y_test)))
  best_weights= model.get_weights()
  

  params_1 =[b + 0.00003 for b in best_weights]
  GlobalMinimaSearch(params_1)


  params_2 =[b - 0.00003 for b in best_weights]
  GlobalMinimaSearch(params_2)
  
 

In [ ]:
GlobalMinimaSearch(mdl.get_weights())

Epoch 1/75
64/64 [==============================] - 9s 12ms/step - loss: 0.0156
Epoch 2/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0089
Epoch 3/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0057
Epoch 4/75
64/64 [==============================] - 1s 12ms/step - loss: 0.0050
Epoch 5/75
64/64 [==============================] - 1s 12ms/step - loss: 0.0045
Epoch 6/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0040
Epoch 7/75
64/64 [==============================] - 1s 12ms/step - loss: 0.0040
Epoch 8/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0037
Epoch 9/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0034
Epoch 10/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0034
Epoch 11/75
64/64 [==============================] - 1s 11ms/step - loss: 0.0033
Epoch 12/75
64/64 [==============================] - 1s 12ms/step - loss: 0.0031
Epoch 13/75
64/64 [==================

In [126]:
print(loss)

[0.053860543908523094, 0.057044855735513864, 0.05648485794152079, 0.056805860393173875, 0.060467348267280785, 0.06202827033358104]


In [127]:
loss_=[]

In [128]:
def GlobalMinimaSearch_(weights,i):
  if len(loss_)>5:
    return
  initial_weights =weights
  model=clone_model(mdl)
  model.set_weights(weights)
  model.compile(optimizer=AdaptiveAdam(), loss='mean_squared_error')
  model.fit(X_train, y_train, epochs=75, batch_size=64)
  y= model.predict(X_test)
  loss_.append(np.sqrt(mean_squared_error(y,y_test)))
  best_weights= model.get_weights()
 

  params_1 =[b + 0.00001*i for b in best_weights]
  GlobalMinimaSearch_(params_1,i+1)


  params_2 =[b - 0.00001*i for b in best_weights]
  GlobalMinimaSearch_(params_2,i+1)
  
  

In [129]:
 GlobalMinimaSearch_(mdl.get_weights(),1)

Epoch 1/75
64/64 [==============================] - 15s 34ms/step - loss: 0.0175
Epoch 2/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0095
Epoch 3/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0056
Epoch 4/75
64/64 [==============================] - 2s 35ms/step - loss: 0.0048
Epoch 5/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0041
Epoch 6/75
64/64 [==============================] - 2s 33ms/step - loss: 0.0042
Epoch 7/75
64/64 [==============================] - 2s 35ms/step - loss: 0.0037
Epoch 8/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0035
Epoch 9/75
64/64 [==============================] - 2s 33ms/step - loss: 0.0034
Epoch 10/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0033
Epoch 11/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0035
Epoch 12/75
64/64 [==============================] - 2s 34ms/step - loss: 0.0033
Epoch 13/75
64/64 [=================

In [130]:
print(loss_)

[0.053829926825430426, 0.05530263444278147, 0.05673249440123113, 0.05790971597768388, 0.05953952926097608, 0.059552224096968434]
